In [1]:
# For data manipulation

import pandas as pd

import numpy as np 

# machine learning classification

from sklearn.svm import SVC

from sklearn.metrics import scorer

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# To plot
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn

# To fetch data

from pandas_datareader import data as pdr

In [13]:
import glob
import pandas as pd
pattern='*.csv'
hong = glob.glob(pattern)
len(hong)

62

In [14]:
hkong = pd.read_csv(hong[0])
hkong = hkong.drop('Adj Close', axis=1)
hkong.columns = ['Date', 'Open' + '_'+str(0), 'High'+'_'+str(0), 'Low' +'_'+str(0), 'Close'+ '_'+str(0), 'Volume'+ '_'+ str(0)]

for date in hkong['Date']:
    hkong['Date'] = pd.to_datetime(hkong['Date'])
hkong =hkong.sort_values(by='Date')

hkong['Volume'+ '_'+ str(0)] = hkong['Volume'+ '_'+ str(0)].fillna(0)
for col in hkong:
    hkong[col] = hkong[col].fillna(method='ffill')
#hkong['Target'] = np.where(hkong['Close'].shift(-1) > hkong['Close'],int(1),int(-1))
hkong.shape

(1234, 6)

In [15]:
hkong['Open' + '_'+str(0)] = ((hkong['Open' + '_'+str(0)] - hkong['Open' + '_'+str(0)].shift(1))/hkong['Open' + '_'+str(0)].shift(1)) *100

hkong['High' + '_'+str(0)] = ((hkong['High' + '_'+str(0)] - hkong['High' + '_'+str(0)].shift(1))/hkong['High' + '_'+str(0)].shift(1)) *100

hkong['Low' + '_'+str(0)] = ((hkong['Low' + '_'+str(0)] - hkong['Low' + '_'+str(0)].shift(1))/hkong['Low' + '_'+str(0)].shift(1)) *100

hkong['Close' + '_'+str(0)] = ((hkong['Close' + '_'+str(0)] - hkong['Close' + '_'+str(0)].shift(1))/hkong['Close' + '_'+str(0)].shift(1)) *100

hkong = hkong.drop(0, axis=0)


print hkong.shape
hkong.head()

#hkong['Open' + '_'+str(0)] = hkong['Open' + '_'+str(0)].drop(hkong['Open' + '_'+str(0)].index[0])


(1233, 6)


,Date,Open_0,High_0,Low_0,Close_0,Volume_0
1,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412
2,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548
3,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634
4,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274
5,2013-03-18,-2.843602,-2.351097,-1.620746,0.646200,2427267


In [16]:
#run only once
for i in range(1,len(hong)):
    df = pd.read_csv(hong[i]) 
    
    if 'Adj Close' in df:
        df = df.drop('Adj Close', axis=1)    
      
    df.columns = ['Date', 'Open' + '_'+str(i), 'High'+'_'+str(i), 'Low' +'_'+str(i), 'Close'+'_'+str(i), 'Volume'+'_'+str(i)]
    
    #formatting date
    for date in df['Date']:
        df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')
    
    #imputing missing values
    df['Volume'+ '_'+ str(i)] = df['Volume'+ '_'+ str(i)].fillna(0)
    for col in df:
        df[col] = df[col].fillna(method='ffill')
    
    df['Open' + '_'+str(i)] = ((df['Open' + '_'+str(i)] - df['Open' + '_'+str(i)].shift(1))/df['Open' + '_'+str(i)].shift(1)) *100

    df['High' + '_'+str(i)] = ((df['High' + '_'+str(i)] - df['High' + '_'+str(i)].shift(1))/df['High' + '_'+str(i)].shift(1)) *100

    df['Low' + '_'+str(i)] = ((df['Low' + '_'+str(i)] - df['Low' + '_'+str(i)].shift(1))/df['Low' + '_'+str(i)].shift(1)) *100

    df['Close' + '_'+str(i)] = ((df['Close' + '_'+str(i)] - df['Close' + '_'+str(i)].shift(1))/df['Close' + '_'+str(i)].shift(1)) *100

    df = df.drop(0, axis=0)
    
    hkong = pd.merge(hkong, df, how='outer', on='Date')

hkong.shape


(1555, 311)

In [12]:
#hkong.tail(25)

In [17]:
for date in hkong['Date']:
    hkong['Date'] = pd.to_datetime(hkong['Date'])
hkong =hkong.sort_values(by='Date')

In [18]:
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Open_60,High_60,Low_60,Close_60,Volume_60,Open_61,High_61,Low_61,Close_61,Volume_61
1233,2013-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,-0.238700,0.531070,-1.382659,-1.038177,117600.0,0.213418,-0.321168,-0.190709,-0.547478,2738000.0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,-1.168739,-2.012475,-0.511300,-0.989981,91000.0,-0.250032,-0.194338,-0.302631,0.010004,2224400.0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,-1.156576,-0.442113,0.002709,0.278674,80200.0,-0.205514,-0.557313,-0.202575,-0.547182,2150200.0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0.600239,1.575284,-0.146681,0.357832,116400.0,-0.159327,-0.034864,-0.268680,-0.305210,2530400.0


In [19]:
for i in range(0,len(hong)):
    hkong['Volume'+ '_'+ str(i)] = hkong['Volume'+ '_'+ str(i)].fillna(0)
for col in hkong.columns:
    hkong[col] = hkong[col].fillna(method='ffill')

In [20]:
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Open_60,High_60,Low_60,Close_60,Volume_60,Open_61,High_61,Low_61,Close_61,Volume_61
1233,2013-03-11,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,-0.238700,0.531070,-1.382659,-1.038177,117600.0,0.213418,-0.321168,-0.190709,-0.547478,2738000.0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,-1.168739,-2.012475,-0.511300,-0.989981,91000.0,-0.250032,-0.194338,-0.302631,0.010004,2224400.0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,-1.156576,-0.442113,0.002709,0.278674,80200.0,-0.205514,-0.557313,-0.202575,-0.547182,2150200.0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0.600239,1.575284,-0.146681,0.357832,116400.0,-0.159327,-0.034864,-0.268680,-0.305210,2530400.0


In [21]:
for i in range(0, len(hong)):
    print hkong['Volume'+'_'+str(i)].isnull().sum()
hkong.isnull().sum()[hkong.isnull().sum() !=0]

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


Open_0      1
High_0      1
Low_0       1
Close_0     1
Open_1      1
High_1      1
Low_1       1
Close_1     1
Open_2      1
High_2      1
Low_2       1
Close_2     1
Open_3      1
High_3      1
Low_3       1
Close_3     1
Open_4      1
High_4      1
Low_4       1
Close_4     1
Open_5      1
High_5      1
Low_5       1
Close_5     1
Open_6      1
High_6      1
Low_6       1
Close_6     1
Open_7      1
High_7      1
           ..
Low_46      2
Close_46    2
Open_47     1
High_47     1
Low_47      1
Close_47    1
Open_49     1
High_49     1
Low_49      1
Close_49    1
Open_51     1
High_51     1
Low_51      1
Close_51    1
Open_52     1
High_52     1
Low_52      1
Close_52    1
Open_57     1
High_57     1
Low_57      1
Close_57    1
Open_60     1
High_60     1
Low_60      1
Close_60    1
Open_61     1
High_61     1
Low_61      1
Close_61    1
Length: 172, dtype: int64

In [22]:
#for col in hkong.columns:
#    hkong[col] = hkong[col].fillna(hkong[col].median(skipna=True))
hkong=hkong.fillna(0)

In [23]:
hkong.isnull().sum()[hkong.isnull().sum() !=0]

Series([], dtype: int64)

In [24]:
for i in range(30):
    hkong['Open'+'_'+str(i)+str(-1)] = hkong['Open'+'_'+str(i)].shift(1)
    hkong['Open'+'_'+str(i)+str(-2)] = hkong['Open'+'_'+str(i)].shift(2)
    hkong['Open'+'_'+str(i)+str(-3)] = hkong['Open'+'_'+str(i)].shift(3)
    hkong['Open'+'_'+str(i)+str(-4)] = hkong['Open'+'_'+str(i)].shift(4)
    hkong['Open'+'_'+str(i)+str(-5)] = hkong['Open'+'_'+str(i)].shift(5)
    hkong['Open'+'_'+str(i)+str(-6)] = hkong['Open'+'_'+str(i)].shift(6)
    hkong['Open'+'_'+str(i)+str(-7)] = hkong['Open'+'_'+str(i)].shift(7)
    hkong['Open'+'_'+str(i)+str(-8)] = hkong['Open'+'_'+str(i)].shift(8)
    hkong['Open'+'_'+str(i)+str(-9)] = hkong['Open'+'_'+str(i)].shift(9)
    hkong['Open'+'_'+str(i)+str(-10)] = hkong['Open'+'_'+str(i)].shift(10)
    
    
    hkong['High'+'_'+str(i)+str(-1)] = hkong['High'+'_'+str(i)].shift(1)
    hkong['High'+'_'+str(i)+str(-2)] = hkong['High'+'_'+str(i)].shift(2)
    hkong['High'+'_'+str(i)+str(-3)] = hkong['High'+'_'+str(i)].shift(3)
    hkong['High'+'_'+str(i)+str(-4)] = hkong['High'+'_'+str(i)].shift(4)
    hkong['High'+'_'+str(i)+str(-5)] = hkong['High'+'_'+str(i)].shift(5)
    hkong['High'+'_'+str(i)+str(-6)] = hkong['High'+'_'+str(i)].shift(6)
    hkong['High'+'_'+str(i)+str(-7)] = hkong['High'+'_'+str(i)].shift(7)
    hkong['High'+'_'+str(i)+str(-8)] = hkong['High'+'_'+str(i)].shift(8)
    hkong['High'+'_'+str(i)+str(-9)] = hkong['High'+'_'+str(i)].shift(9)
    hkong['High'+'_'+str(i)+str(-10)] = hkong['High'+'_'+str(i)].shift(10)
    
    hkong['Low'+'_'+str(i)+str(-1)] = hkong['Low'+'_'+str(i)].shift(1)
    hkong['Low'+'_'+str(i)+str(-2)] = hkong['Low'+'_'+str(i)].shift(2)
    hkong['Low'+'_'+str(i)+str(-3)] = hkong['Low'+'_'+str(i)].shift(3)
    hkong['Low'+'_'+str(i)+str(-4)] = hkong['Low'+'_'+str(i)].shift(4)
    hkong['Low'+'_'+str(i)+str(-5)] = hkong['Low'+'_'+str(i)].shift(5)
    hkong['Low'+'_'+str(i)+str(-6)] = hkong['Low'+'_'+str(i)].shift(6)
    hkong['Low'+'_'+str(i)+str(-7)] = hkong['Low'+'_'+str(i)].shift(7)
    hkong['Low'+'_'+str(i)+str(-8)] = hkong['Low'+'_'+str(i)].shift(8)
    hkong['Low'+'_'+str(i)+str(-9)] = hkong['Low'+'_'+str(i)].shift(9)
    hkong['Low'+'_'+str(i)+str(-10)] = hkong['Low'+'_'+str(i)].shift(10)
    
    hkong['Close'+'_'+str(i)+str(-1)] = hkong['Close'+'_'+str(i)].shift(1)
    hkong['Close'+'_'+str(i)+str(-2)] = hkong['Close'+'_'+str(i)].shift(2)
    hkong['Close'+'_'+str(i)+str(-3)] = hkong['Close'+'_'+str(i)].shift(3)
    hkong['Close'+'_'+str(i)+str(-4)] = hkong['Close'+'_'+str(i)].shift(4)
    hkong['Close'+'_'+str(i)+str(-5)] = hkong['Close'+'_'+str(i)].shift(5)
    hkong['Close'+'_'+str(i)+str(-6)] = hkong['Close'+'_'+str(i)].shift(6)
    hkong['Close'+'_'+str(i)+str(-7)] = hkong['Close'+'_'+str(i)].shift(7)
    hkong['Close'+'_'+str(i)+str(-8)] = hkong['Close'+'_'+str(i)].shift(8)
    hkong['Close'+'_'+str(i)+str(-9)] = hkong['Close'+'_'+str(i)].shift(9)
    hkong['Close'+'_'+str(i)+str(-10)] = hkong['Close'+'_'+str(i)].shift(10)
    
    hkong['Volume'+'_'+str(i)+str(-1)] = hkong['Volume'+'_'+str(i)].shift(1)
    hkong['Volume'+'_'+str(i)+str(-2)] = hkong['Volume'+'_'+str(i)].shift(2)
    hkong['Volume'+'_'+str(i)+str(-3)] = hkong['Volume'+'_'+str(i)].shift(3)
    hkong['Volume'+'_'+str(i)+str(-4)] = hkong['Volume'+'_'+str(i)].shift(4)
    hkong['Volume'+'_'+str(i)+str(-5)] = hkong['Volume'+'_'+str(i)].shift(5)
    hkong['Volume'+'_'+str(i)+str(-6)] = hkong['Volume'+'_'+str(i)].shift(6)
    hkong['Volume'+'_'+str(i)+str(-7)] = hkong['Volume'+'_'+str(i)].shift(7)
    hkong['Volume'+'_'+str(i)+str(-8)] = hkong['Volume'+'_'+str(i)].shift(8)
    hkong['Volume'+'_'+str(i)+str(-9)] = hkong['Volume'+'_'+str(i)].shift(9)
    hkong['Volume'+'_'+str(i)+str(-10)] = hkong['Volume'+'_'+str(i)].shift(10)

hkong = hkong.fillna(0)
hkong.shape

(1555, 1811)

In [25]:
for i in range(30):
    hkong['Target'+'_'+str(i)] = np.where(hkong['Close'+'_'+str(i)]>0.5,1,0)
    
hkong.shape

(1555, 1841)

In [26]:
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
1233,2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,0,0,0,1,0,0,0,0,0,0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,0,0,0,0,0,0,0,0,0,0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,0,1,0,0,0,0,1,1,0,0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0,0,0,0,0,0,0,0,0,0


In [27]:
hkong.index = hkong.Date
hkong = hkong.drop('Date', axis=1)
hkong.shape

(1555, 1840)

In [28]:
hkong.head()

,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,Volume_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
Date,,,,,,,,,,,,,,,,,,,,,
2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,23618156.0,...,0,0,0,1,0,0,0,0,0,0
2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,31644955.0,...,0,0,0,0,0,0,0,0,0,0
2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,21286152.0,...,0,1,0,0,0,0,1,1,0,0
2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,44707880.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
hkong.to_csv('E:/hkongpercent-10-best_currency.csv')

In [4]:
#whenever open fresh, start from here
hkong = pd.read_csv('E:/hangseng/hkongpercent-10-best_currency.csv')   
print hkong.shape
hkong.head()

(1555, 1841)


,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
0,2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,0,0,0,1,0,0,0,0,0,0
2,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,0,0,0,0,0,0,0,0,0,0
3,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,0,1,0,0,0,0,1,1,0,0
4,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0,0,0,0,0,0,0,0,0,0


In [5]:
hkong.index = hkong.Date
hkong = hkong.drop('Date', axis=1)
hkong.shape

(1555, 1840)

In [6]:
target=hkong.iloc[:, 1810:].columns
target

Index([u'Target_0', u'Target_1', u'Target_2', u'Target_3', u'Target_4',
       u'Target_5', u'Target_6', u'Target_7', u'Target_8', u'Target_9',
       u'Target_10', u'Target_11', u'Target_12', u'Target_13', u'Target_14',
       u'Target_15', u'Target_16', u'Target_17', u'Target_18', u'Target_19',
       u'Target_20', u'Target_21', u'Target_22', u'Target_23', u'Target_24',
       u'Target_25', u'Target_26', u'Target_27', u'Target_28', u'Target_29'],
      dtype='object')

In [7]:
X = hkong.iloc[:, :1810]

sc = StandardScaler()
X = sc.fit_transform(X)
X.shape

(1555, 1810)

In [8]:
scores = []
test_acc =[]

for col in target:
    y = hkong[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = LogisticRegression()

    cv_score = np.mean(cross_val_score(classifier, X_test, y_test, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    #print('CV score for class {} is {}'.format(col, cv_score))

    classifier.fit(X_train, y_train)
    accuracy_train = accuracy_score(y_train, classifier.predict(X_train))

    accuracy_test = accuracy_score(y_test, classifier.predict(X_test))
    test_acc.append(accuracy_test*100)
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))
print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_LR_10 =test_acc
print test_acc_LR_10

Total CV score is 0.796881361026
Mean Test Accuracy is  79.13%
[77.52, 80.09, 77.3, 79.87, 77.94, 76.87, 78.16, 78.8, 74.73, 75.59, 78.59, 78.37, 78.59, 80.3, 79.23, 80.73, 80.09, 79.87, 77.09, 76.45, 80.94, 74.73, 80.94, 81.8, 77.52, 75.8, 79.66, 79.87, 80.3, 96.15]


In [9]:
scores = []
test_acc =[]
for col in target:
    y = hkong[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = SVC()

    cv_score = np.mean(cross_val_score(classifier, X_test, y_test, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    #print('CV score for class {} is {}'.format(col, cv_score))

    classifier.fit(X_train, y_train)
    accuracy_train = accuracy_score(y_train, classifier.predict(X_train))

    accuracy_test = accuracy_score(y_test, classifier.predict(X_test))
    test_acc.append(accuracy_test*100)
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))
print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_SVC_10 =test_acc
print test_acc_SVC_10

Total CV score is 0.786630600525
Mean Test Accuracy is  77.06%
[70.66, 79.23, 72.16, 80.3, 70.88, 76.45, 77.94, 76.02, 73.45, 76.45, 73.23, 78.16, 76.45, 76.66, 80.09, 76.87, 74.09, 80.51, 72.59, 79.23, 79.23, 74.52, 75.59, 77.73, 79.44, 74.52, 80.09, 77.73, 74.09, 97.43]


In [10]:
scores = []
test_acc =[]
for col in target:
    y = hkong[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = RandomForestClassifier()

    cv_score = np.mean(cross_val_score(classifier, X_test, y_test, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    #print('CV score for class {} is {}'.format(col, cv_score))

    classifier.fit(X_train, y_train)
    accuracy_train = accuracy_score(y_train, classifier.predict(X_train))

    accuracy_test = accuracy_score(y_test, classifier.predict(X_test))
    test_acc.append(accuracy_test*100)
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))
print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_RFC_10 =test_acc
print test_acc_RFC_10

Total CV score is 0.868074273338
Mean Test Accuracy is  85.39%
[72.81, 85.01, 79.01, 89.94, 87.15, 90.36, 83.73, 80.51, 78.59, 92.08, 87.15, 82.66, 89.51, 88.01, 85.87, 89.51, 76.23, 87.15, 77.73, 80.94, 82.66, 86.3, 87.79, 89.94, 83.73, 88.87, 90.15, 82.66, 87.79, 97.86]


In [11]:
scores = []
test_acc =[]
for col in target:
    y = hkong[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = GradientBoostingClassifier()

    cv_score = np.mean(cross_val_score(classifier, X_test, y_test, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(col, cv_score))

    classifier.fit(X_train, y_train)
    accuracy_train = accuracy_score(y_train, classifier.predict(X_train))

    accuracy_test = accuracy_score(y_test, classifier.predict(X_test))
    test_acc.append(accuracy_test*100)
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))
print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_GBC_10 =test_acc
print test_acc_GBC_10

CV score for class Target_0 is 1.0
CV score for class Target_1 is 1.0
CV score for class Target_2 is 0.996666666667
CV score for class Target_3 is 1.0
CV score for class Target_4 is 1.0
CV score for class Target_5 is 0.996428571429
CV score for class Target_6 is 0.996875
CV score for class Target_7 is 0.998387096774
CV score for class Target_8 is 0.996875
CV score for class Target_9 is 0.995104166667
CV score for class Target_10 is 1.0
CV score for class Target_11 is 1.0
CV score for class Target_12 is 0.992083333333
CV score for class Target_13 is 0.997368421053
CV score for class Target_14 is 1.0
CV score for class Target_15 is 1.0
CV score for class Target_16 is 0.998387096774
CV score for class Target_17 is 1.0
CV score for class Target_18 is 0.998387096774
CV score for class Target_19 is 1.0
CV score for class Target_20 is 1.0
CV score for class Target_21 is 0.998484848485
CV score for class Target_22 is 1.0
CV score for class Target_23 is 0.995334685598
CV score for class Target_

In [14]:
testacc_df = pd.DataFrame([hong[:30], test_acc_LR_10, test_acc_SVC_10, test_acc_RFC_10, test_acc_GBC_10]).T
testacc_df.columns = ['Stock Components', 'Logistic Regression % (0.5) (10-)', 'SVM % (0.5) (10-)', 'Random Forest % (0.5) (10-)', 'Gradient Boosting % (0.5) (10-)']
testacc_df['Stock Components'] = map(lambda x:x.replace('.csv', ''), testacc_df['Stock Components'])
testacc_df

,Stock Components,Logistic Regression % (0.5) (10-),SVM % (0.5) (10-),Random Forest % (0.5) (10-),Gradient Boosting % (0.5) (10-)
0,AAC Technologies Holdings Inc. (2018.HK),77.52,70.66,72.81,100
1,AIA Group Limited (1299.HK),80.09,79.23,85.01,100
2,BOC Hong Kong (Holdings) Limited (2388.HK),77.3,72.16,79.01,100
3,China Life Insurance Company Limited (2628.HK),79.87,80.3,89.94,100
4,China Mengniu Dairy Company Limited (2319.HK),77.94,70.88,87.15,100
5,China Mobile Limited (0941.HK),76.87,76.45,90.36,100
6,China Petroleum & Chemical Corporation(0386.HK),78.16,77.94,83.73,100
7,China Resources Land Limited (1109.HK),78.8,76.02,80.51,100
8,China Resources Power Holdings Company Limited...,74.73,73.45,78.59,100
9,CITIC Limited (0267.HK),75.59,76.45,92.08,100


In [15]:
testacc_df.to_csv('E:/span10_0p5.csv', index=False)